In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from tqdm.notebook import tqdm
from constants import Constants
from kleague_parser import KLeagueParser

base_path = os.path.join(os.path.dirname(os.getcwd()))
print(f"Base path: {base_path}")

pd.set_option('display.width', 250)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)

%matplotlib inline
mpl.rcParams['animation.embed_limit'] = 100

Base path: /root/express-v2


### Counting the Number of Matches Satisfying a Given Conditions

In [2]:
team_list = ["울산", "전북", "서울", "대전", "제주", "김천", "김포", "아산", "안산", "광주", "강원", "포항", "대구", "인천", "수원F"]

season = "2024"
match_ids = np.sort([int(f) for f in os.listdir(f"{base_path}/data/bepro/raw") if f[0] != "."])
constants = Constants()

count = 0
for i, match_id in enumerate(match_ids):
    # metadata_path = f"data/{season}/raw_data/{match_id}/{match_id}-match_info.json"
    metadata_path = f"{base_path}/data/bepro/raw/{match_id}/{match_id}_metadata.json"
    with open(metadata_path) as f:
        metadata = json.load(f)
        home_team = constants.TEAM_DICT[metadata["home_team"]["team_name"]]
        away_team = constants.TEAM_DICT[metadata["away_team"]["team_name"]]
        
        if home_team in team_list and away_team in team_list:
            count += 1
            print(f"[{i}] {match_id}: {home_team} vs {away_team}")
        else:
            raise ValueError(f"Invalid team names: {home_team}, {away_team}")

count

[0] 126285: 광주 vs 서울
[1] 126293: 광주 vs 강원
[2] 126298: 포항 vs 광주
[3] 126306: 광주 vs 대구
[4] 126309: 광주 vs 인천
[5] 126315: 김천 vs 광주
[6] 126319: 전북 vs 광주
[7] 126325: 광주 vs 울산
[8] 126332: 광주 vs 수원F
[9] 126341: 제주 vs 광주
[10] 126348: 광주 vs 대전
[11] 126350: 대구 vs 광주
[12] 126356: 광주 vs 전북
[13] 126364: 인천 vs 광주
[14] 126367: 광주 vs 포항
[15] 126378: 서울 vs 광주
[16] 126380: 광주 vs 김천
[17] 126386: 대전 vs 광주
[18] 126391: 수원F vs 광주
[19] 126401: 광주 vs 제주
[20] 126408: 강원 vs 광주
[21] 126411: 울산 vs 광주
[22] 126418: 광주 vs 인천
[23] 126424: 대구 vs 광주
[24] 126429: 광주 vs 수원F
[25] 126433: 전북 vs 광주
[26] 126444: 강원 vs 광주
[27] 126448: 광주 vs 울산
[28] 126455: 대전 vs 광주
[29] 126458: 광주 vs 포항
[30] 126466: 광주 vs 제주
[31] 126473: 김천 vs 광주
[32] 126476: 광주 vs 서울
[33] 153364: 광주 vs 대구
[34] 153373: 인천 vs 광주
[35] 153379: 광주 vs 대전
[36] 153381: 수원F vs 강원
[37] 153385: 제주 vs 광주
[38] 153387: 울산 vs 수원F
[39] 153390: 광주 vs 전북


40

In [48]:
match_id = 126424#match_ids[1]
print(f"Processing match ID: {match_id}")

with open(f"{base_path}/data/bepro/processed/{match_id}/{match_id}_processed_dict.pkl", "rb") as f:
    match_dict = pickle.load(f)
    events = match_dict['event_df']
    teams_dict = match_dict['teams']
    metadata = match_dict['meta_data']
    tracking = match_dict['tracking_df']

Processing match ID: 126424


In [49]:
tracking

,game_id,period_id,timestamp,frame_id,ball_state,ori_ball_owning_team_id,x,y,z,vx,vy,vz,v,ax,ay,az,a,id,team_id,position_name,ball_owning_team_id,is_ball_carrier
0,126424,1.0,0 days 00:00:00.621000,15,alive,4648,-18.8076,12.7844,0.0,-0.176709,0.185511,0.0,0.256203,0.305389,-0.311730,0.0,0.436392,145703,4648,CB,4648,False
1,126424,1.0,0 days 00:00:00.621000,15,alive,4648,10.9481,16.4773,0.0,0.380973,0.246792,0.0,0.453924,-0.261936,0.033493,0.0,0.264068,250055,4644,RWB,4648,False
2,126424,1.0,0 days 00:00:00.621000,15,alive,4648,47.5261,-0.8557,0.0,-0.429694,-0.121898,0.0,0.446649,-0.079109,-0.080602,0.0,0.112938,250060,4644,GK,4648,False
3,126424,1.0,0 days 00:00:00.621000,15,alive,4648,13.9781,-22.3153,0.0,-0.000769,0.002639,0.0,0.002749,-0.056704,-0.023303,0.0,0.061306,250076,4644,LWB,4648,False
4,126424,1.0,0 days 00:00:00.621000,15,alive,4648,-47.6475,0.6478,0.0,-0.703864,0.013661,0.0,0.703997,-0.485115,-0.078116,0.0,0.491364,250079,4648,GK,4648,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3546641,126424,2.0,0 days 01:40:04.205000,150105,alive,4644,41.3183,-5.2598,0.0,4.098690,-0.154004,0.0,4.101582,0.509364,-1.258938,0.0,1.358079,62086,4644,CB,4644,False
3546642,126424,2.0,0 days 01:40:04.205000,150105,alive,4644,37.1069,-15.2927,0.0,0.225230,3.594709,0.0,3.601759,-0.168122,-0.585722,0.0,0.609372,62112,4644,LW,4644,False
3546643,126424,2.0,0 days 01:40:04.205000,150105,alive,4644,34.1823,-27.1252,0.0,-0.512353,1.277545,0.0,1.376454,0.679728,0.237815,0.0,0.720130,62119,4644,RW,4644,False
3546644,126424,2.0,0 days 01:40:04.205000,150105,alive,4644,46.8050,1.7155,0.0,1.435471,0.598111,0.0,1.555093,-1.209735,-0.944719,0.0,1.534911,62365,4648,CB,4644,False


In [50]:
events

,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,player_shirt_number,player_name,events,x,y,to_x,to_y,attack_direction
0,Half,1st Half,0,2700000,0,533,Gwangju FC,99,Beka Mikeltadze,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4976,0.5045,0.4236,0.6365,RIGHT
1,Half,1st Half,0,2700000,0,1667,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes Received', 'property':...",0.4236,0.6365,NaN,NaN,RIGHT
2,Half,1st Half,0,2700000,0,5167,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4464,0.6543,0.3413,0.7579,RIGHT
3,Half,1st Half,0,2700000,0,6500,Gwangju FC,18,Yool Heo,"[{'event_name': 'Passes Received', 'property':...",0.3413,0.7579,NaN,NaN,RIGHT
4,Half,1st Half,0,2700000,0,15021,Gwangju FC,18,Yool Heo,"[{'name': 'VHIR', 'property': {'duration': 500...",0.3500,0.5752,0.3501,0.5339,RIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,Half,2nd Half,1,2700000,2700000,5999333,Daegu FC,11,Cesinha,"[{'event_name': 'Passes Received', 'property':...",0.8861,0.1069,NaN,NaN,RIGHT
1833,Half,2nd Half,1,2700000,2700000,6000600,Daegu FC,11,Cesinha,"[{'event_name': 'Passes', 'property': {'Outcom...",0.8691,0.1376,0.7890,0.2495,RIGHT
1834,Half,2nd Half,1,2700000,2700000,6002367,Daegu FC,2,Jaewon Hwang,"[{'event_name': 'Crosses', 'property': {'Outco...",0.7890,0.2495,0.9670,0.6118,RIGHT
1835,Half,2nd Half,1,2700000,2700000,6004200,Daegu FC,17,Italo Carvalho,"[{'event_name': 'Duels', 'property': {'Type': ...",0.9670,0.6118,NaN,NaN,RIGHT


### Parsing Match Data

In [51]:
# total_mismatches = []
# for match_id in tqdm(match_ids):

#     with open(f"{base_path}/data/bepro/processed/{match_id}/{match_id}_processed_dict.pkl", "rb") as f:
#         match_dict = pickle.load(f)
#         events = match_dict['event_df']
#         teams_dict = match_dict['teams']
#         metadata = match_dict['meta_data']
#         tracking = match_dict['tracking_df']

#     parser = KLeagueParser(match_id, data_dir=f"{base_path}/data/bepro/processed")
#     parser.parse_events()
#     parser.parse_ball_xy()

#     parser.combine_events_and_ball_xy()
#     parser.parse_player_xy()
#     parser.combine_player_and_ball_xy()

#     mismatch = parser.mislabels

#     if mismatch is None:
#         print(f"No mismatches found for match ID: {match_id}")
#         continue
#     mismatch["match_id"] = match_id
#     total_mismatches.append(mismatch)

# total_mismatches_df = pd.concat(total_mismatches, ignore_index=True)
# total_mismatches_df

In [52]:
parser = KLeagueParser(match_id, data_dir=f"{base_path}/data/bepro/processed")
parser.parse_events()
parser.parse_ball_xy()

parser.combine_events_and_ball_xy()
parser.events

,event_id,session,time,home_away,player_code,event_types,x,y
0,0,1,0.533,A,A99,"[{'event_name': 'Passes', 'property': {'Outcom...",-1.140000,1.335000
1,1,1,1.667,A,A10,"[{'event_name': 'Passes Received', 'property':...",-8.019254,9.289130
2,2,1,5.167,A,A10,"[{'event_name': 'Passes', 'property': {'Outcom...",-5.849358,10.635242
3,3,1,6.500,A,A18,"[{'event_name': 'Passes Received', 'property':...",-16.667438,17.517902
5,5,1,15.933,A,A18,"[{'event_name': 'Passes', 'property': {'Outcom...",-16.127147,-0.773783
...,...,...,...,...,...,...,...,...
3171,3433,2,3299.333,H,H11,"[{'event_name': 'Passes Received', 'property':...",40.505148,-26.695085
3172,3434,2,3300.600,H,H11,"[{'event_name': 'Passes', 'property': {'Outcom...",38.751500,-24.641400
3173,3435,2,3302.367,H,H02,"[{'event_name': 'Crosses', 'property': {'Outco...",30.352689,-17.070160
3174,3436,2,3304.200,H,H17,"[{'event_name': 'Duels', 'property': {'Type': ...",49.186700,7.717600


In [53]:
parser.ball_xy

,session,frame,time,x,y
0,1,15,0.621,-1.140000,1.335000
1,1,16,0.661,-1.409191,1.646046
2,1,17,0.701,-1.675447,1.953743
3,1,18,0.741,-1.940143,2.259499
4,1,19,0.781,-2.209851,2.571141
...,...,...,...,...,...
154477,2,150101,3304.045,47.402049,5.453247
154478,2,150102,3304.085,47.804526,5.983644
154479,2,150103,3304.125,48.217933,6.528349
154480,2,150104,3304.165,48.612191,7.052044


In [54]:
parser.events

,event_id,session,time,home_away,player_code,event_types,x,y
0,0,1,0.533,A,A99,"[{'event_name': 'Passes', 'property': {'Outcom...",-1.140000,1.335000
1,1,1,1.667,A,A10,"[{'event_name': 'Passes Received', 'property':...",-8.019254,9.289130
2,2,1,5.167,A,A10,"[{'event_name': 'Passes', 'property': {'Outcom...",-5.849358,10.635242
3,3,1,6.500,A,A18,"[{'event_name': 'Passes Received', 'property':...",-16.667438,17.517902
5,5,1,15.933,A,A18,"[{'event_name': 'Passes', 'property': {'Outcom...",-16.127147,-0.773783
...,...,...,...,...,...,...,...,...
3171,3433,2,3299.333,H,H11,"[{'event_name': 'Passes Received', 'property':...",40.505148,-26.695085
3172,3434,2,3300.600,H,H11,"[{'event_name': 'Passes', 'property': {'Outcom...",38.751500,-24.641400
3173,3435,2,3302.367,H,H02,"[{'event_name': 'Crosses', 'property': {'Outco...",30.352689,-17.070160
3174,3436,2,3304.200,H,H17,"[{'event_name': 'Duels', 'property': {'Type': ...",49.186700,7.717600


In [55]:
parser.parse_player_xy()
if parser.player_xy is not None:
    parser.combine_player_and_ball_xy()
    # parser.split_into_episodes()
    # parser.calc_running_features()
    # parser.save()
    
parser.traces

,frame,event_id,session,phase,player_code,event_types,ball_x,ball_y,time,A01_x,A01_y,A06_x,A06_y,A07_x,A07_y,...,H14_y,H17_x,H17_y,H21_x,H21_y,H30_x,H30_y,H32_x,H32_y,H33_x,H33_y,H40_x,H40_y,H99_x,H99_y
0,0,0.0,1,1,A99,"[{'event_name': 'Passes', 'property': {'Outcom...",-1.1400,1.3350,0.621,-47.647500,0.6478,-19.383300,-7.425500,NaN,NaN,...,-6.021800,NaN,NaN,47.526100,-0.855700,0.631400,24.315600,NaN,NaN,13.978100,-22.315300,14.131100,-0.566600,NaN,NaN
1,1,NaN,1,1,NaN,NaN,NaN,NaN,0.661,-47.677349,0.6478,-19.383300,-7.442515,NaN,NaN,...,-6.014971,NaN,NaN,47.508733,-0.860681,0.615673,24.285596,NaN,NaN,13.978100,-22.315288,14.116705,-0.570591,NaN,NaN
2,2,NaN,1,1,NaN,NaN,NaN,NaN,0.701,-47.706949,0.6478,-19.383300,-7.458575,NaN,NaN,...,-6.006293,NaN,NaN,47.491272,-0.865829,0.600574,24.255322,NaN,NaN,13.978020,-22.315200,14.102972,-0.574361,NaN,NaN
3,3,NaN,1,1,NaN,NaN,NaN,NaN,0.741,-47.735831,0.6478,-19.383300,-7.474163,NaN,NaN,...,-5.995666,NaN,NaN,47.473517,-0.871249,0.586213,24.224770,NaN,NaN,13.977629,-22.315134,14.089926,-0.577825,NaN,NaN
4,4,NaN,1,1,NaN,NaN,NaN,NaN,0.781,-47.764295,0.6478,-19.383300,-7.490028,NaN,NaN,...,-5.982694,NaN,NaN,47.454831,-0.877188,0.572216,24.193526,NaN,NaN,13.977099,-22.315100,14.077142,-0.580973,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154492,154488,NaN,2,7,NaN,NaN,NaN,NaN,3304.085,50.762141,1.1844,46.626964,1.639410,36.559509,4.421682,...,NaN,47.094151,6.364951,-4.685366,-0.714449,NaN,NaN,34.247067,-27.277813,NaN,NaN,26.034548,6.406356,43.327509,0.260160
154493,154489,NaN,2,7,NaN,NaN,NaN,NaN,3304.125,50.796873,1.1844,46.689578,1.667169,36.574796,4.503191,...,NaN,47.168583,6.268893,-4.724101,-0.703974,NaN,NaN,34.223880,-27.227239,NaN,NaN,26.076378,6.377710,43.402393,0.282412
154494,154490,NaN,2,7,NaN,NaN,NaN,NaN,3304.165,50.822829,1.1844,46.748123,1.692241,36.588766,4.585025,...,NaN,47.238592,6.175580,-4.761570,-0.693466,NaN,NaN,34.202588,-27.177252,NaN,NaN,26.116911,6.349781,43.476948,0.305798
154495,154491,3436.0,2,7,H17,"[{'event_name': 'Duels', 'property': {'Type': ...",49.1867,7.7176,3304.205,50.841000,1.1844,46.805000,1.715500,36.601800,4.671000,...,NaN,47.307500,6.081100,-4.799400,-0.682400,NaN,NaN,34.182300,-27.125200,NaN,NaN,26.158200,6.321500,43.553800,0.330600


In [56]:
parser.mislabels

,,session,time,annotated,corrected,ball_error
frame,event_id,,,,,
8186,224.0,1,328.061,H33,H09,28.825927
10431,264.0,1,417.861,A30,A11,9.004828
15294,385.0,1,612.381,A06,A99,25.857844
45272,1019.0,1,1813.981,A14,A27,24.788684
50828,1140.0,1,2036.221,A99,A10,15.624990
58771,1291.0,1,2353.941,H06,H02,9.882519
78982,1803.0,2,283.845,A06,A27,17.159629
78996,1804.0,2,284.405,A06,A27,18.884727
81092,1868.0,2,368.245,H11,H17,18.843808


In [62]:
parser.events

,frame,event_id,session,time,home_away,player_code,event_types,x,y,phase
0,0,0,1,0.533,A,A99,"[{'event_name': 'Passes', 'property': {'Outcom...",-1.140000,1.335000,1
1,26,1,1,1.667,A,A10,"[{'event_name': 'Passes Received', 'property':...",-8.019254,9.289130,1
2,114,2,1,5.167,A,A10,"[{'event_name': 'Passes', 'property': {'Outcom...",-5.849358,10.635242,1
3,147,3,1,6.500,A,A18,"[{'event_name': 'Passes Received', 'property':...",-16.667438,17.517902,1
4,383,5,1,15.933,A,A18,"[{'event_name': 'Passes', 'property': {'Outcom...",-16.127147,-0.773783,1
...,...,...,...,...,...,...,...,...,...,...
2040,154369,3433,2,3299.333,H,H11,"[{'event_name': 'Passes Received', 'property':...",40.505148,-26.695085,7
2041,154401,3434,2,3300.600,H,H11,"[{'event_name': 'Passes', 'property': {'Outcom...",38.751500,-24.641400,7
2042,154445,3435,2,3302.367,H,H02,"[{'event_name': 'Crosses', 'property': {'Outco...",30.352689,-17.070160,7
2043,154491,3436,2,3304.200,H,H17,"[{'event_name': 'Duels', 'property': {'Type': ...",49.186700,7.717600,7


In [77]:
team_sheets = pd.concat([teams_dict["Home"], teams_dict["Away"]], ignore_index=True)
player_code_to_player_id = {f"{row['team'][0]}{int(row['jID']):02d}": row['pID'] for _, row in team_sheets.iterrows()}
player_code_to_player_id

{'H01': '62082',
 'H10': '62098',
 'H17': '530662',
 'H02': '250055',
 'H21': '250060',
 'H32': '62119',
 'H22': '62105',
 'H07': '62086',
 'H99': '342722',
 'H40': '500515',
 'H09': '62114',
 'H06': '500520',
 'H33': '250076',
 'H05': '500505',
 'H14': '325355',
 'H03': '145716',
 'H11': '62112',
 'H30': '259766',
 'H74': '187277',
 'H04': '529335',
 'A14': '250101',
 'A30': '500139',
 'A28': '500141',
 'A09': '500134',
 'A07': '359624',
 'A27': '500140',
 'A10': '77414',
 'A20': '250102',
 'A47': '360166',
 'A24': '360389',
 'A99': '408383',
 'A18': '145703',
 'A40': '345465',
 'A11': '500135',
 'A06': '62365',
 'A01': '250079',
 'A31': '250105',
 'A88': '500136',
 'A96': '503227',
 'A22': '250100'}

In [82]:
e = parser.events
e["player_id"] = e["player_code"].map(player_code_to_player_id)
e = e[["event_id", "player_id"]]
e

,event_id,player_id
0,0,408383
1,1,77414
2,2,77414
3,3,145703
4,5,145703
...,...,...
2040,3433,62112
2041,3434,62112
2042,3435,250055
2043,3436,530662


In [87]:
raw_events.columns

Index(['index', 'period_type', 'period_name', 'period_order', 'period_duration', 'period_start_time', 'event_time', 'team_name', 'player_shirt_number', 'player_name', 'events', 'x', 'y', 'to_x', 'to_y', 'attack_direction'], dtype='object')

In [94]:
events["event_id"] = range(len(events))
events

,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,player_shirt_number,player_name,events,x,y,to_x,to_y,attack_direction,event_id
0,Half,1st Half,0,2700000,0,533,Gwangju FC,99,Beka Mikeltadze,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4976,0.5045,0.4236,0.6365,RIGHT,0
1,Half,1st Half,0,2700000,0,1667,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes Received', 'property':...",0.4236,0.6365,NaN,NaN,RIGHT,1
2,Half,1st Half,0,2700000,0,5167,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4464,0.6543,0.3413,0.7579,RIGHT,2
3,Half,1st Half,0,2700000,0,6500,Gwangju FC,18,Yool Heo,"[{'event_name': 'Passes Received', 'property':...",0.3413,0.7579,NaN,NaN,RIGHT,3
4,Half,1st Half,0,2700000,0,15021,Gwangju FC,18,Yool Heo,"[{'name': 'VHIR', 'property': {'duration': 500...",0.3500,0.5752,0.3501,0.5339,RIGHT,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,Half,2nd Half,1,2700000,2700000,5999333,Daegu FC,11,Cesinha,"[{'event_name': 'Passes Received', 'property':...",0.8861,0.1069,NaN,NaN,RIGHT,3433
1833,Half,2nd Half,1,2700000,2700000,6000600,Daegu FC,11,Cesinha,"[{'event_name': 'Passes', 'property': {'Outcom...",0.8691,0.1376,0.7890,0.2495,RIGHT,3434
1834,Half,2nd Half,1,2700000,2700000,6002367,Daegu FC,2,Jaewon Hwang,"[{'event_name': 'Crosses', 'property': {'Outco...",0.7890,0.2495,0.9670,0.6118,RIGHT,3435
1835,Half,2nd Half,1,2700000,2700000,6004200,Daegu FC,17,Italo Carvalho,"[{'event_name': 'Duels', 'property': {'Type': ...",0.9670,0.6118,NaN,NaN,RIGHT,3436


In [ ]:
raw_events = events.reset_index(drop=True)
raw_events = raw_events[['index', 'period_type', 'period_name', 'period_order', 
                         'period_duration', 'period_start_time', 'event_time', 
                         'team_name', 'events', 'x', 'y', 'to_x', 'to_y', 'attack_direction']]
raw_events

,index,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,events,x,y,to_x,to_y,attack_direction
0,0,Half,1st Half,0,2700000,0,533,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4976,0.5045,0.4236,0.6365,RIGHT
1,1,Half,1st Half,0,2700000,0,1667,Gwangju FC,"[{'event_name': 'Passes Received', 'property':...",0.4236,0.6365,NaN,NaN,RIGHT
2,2,Half,1st Half,0,2700000,0,5167,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4464,0.6543,0.3413,0.7579,RIGHT
3,3,Half,1st Half,0,2700000,0,6500,Gwangju FC,"[{'event_name': 'Passes Received', 'property':...",0.3413,0.7579,NaN,NaN,RIGHT
4,4,Half,1st Half,0,2700000,0,15021,Gwangju FC,"[{'name': 'VHIR', 'property': {'duration': 500...",0.3500,0.5752,0.3501,0.5339,RIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433,1832,Half,2nd Half,1,2700000,2700000,5999333,Daegu FC,"[{'event_name': 'Passes Received', 'property':...",0.8861,0.1069,NaN,NaN,RIGHT
3434,1833,Half,2nd Half,1,2700000,2700000,6000600,Daegu FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.8691,0.1376,0.7890,0.2495,RIGHT
3435,1834,Half,2nd Half,1,2700000,2700000,6002367,Daegu FC,"[{'event_name': 'Crosses', 'property': {'Outco...",0.7890,0.2495,0.9670,0.6118,RIGHT
3436,1835,Half,2nd Half,1,2700000,2700000,6004200,Daegu FC,"[{'event_name': 'Duels', 'property': {'Type': ...",0.9670,0.6118,NaN,NaN,RIGHT


In [89]:
e.merge(raw_events, left_on="event_id", right_on="index", how="left")

,event_id,player_id,index,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,events,x,y,to_x,to_y,attack_direction
0,0,408383,0.0,Half,1st Half,0.0,2700000.0,0.0,533.0,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4976,0.5045,0.4236,0.6365,RIGHT
1,0,408383,0.0,Half,2nd Half,1.0,2700000.0,2700000.0,2700300.0,Daegu FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4977,0.5119,0.4088,0.5073,RIGHT
2,1,77414,1.0,Half,1st Half,0.0,2700000.0,0.0,1667.0,Gwangju FC,"[{'event_name': 'Passes Received', 'property':...",0.4236,0.6365,NaN,NaN,RIGHT
3,1,77414,1.0,Half,2nd Half,1.0,2700000.0,2700000.0,2701367.0,Daegu FC,"[{'event_name': 'Passes Received', 'property':...",0.4088,0.5073,NaN,NaN,RIGHT
4,2,77414,2.0,Half,1st Half,0.0,2700000.0,0.0,5167.0,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4464,0.6543,0.3413,0.7579,RIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,3433,62112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2997,3434,62112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2998,3435,250055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2999,3436,530662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
events.reset_index(drop=False)

,index,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,player_shirt_number,player_name,events,x,y,to_x,to_y,attack_direction
0,0,Half,1st Half,0,2700000,0,533,Gwangju FC,99,Beka Mikeltadze,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4976,0.5045,0.4236,0.6365,RIGHT
1,1,Half,1st Half,0,2700000,0,1667,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes Received', 'property':...",0.4236,0.6365,NaN,NaN,RIGHT
2,2,Half,1st Half,0,2700000,0,5167,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4464,0.6543,0.3413,0.7579,RIGHT
3,3,Half,1st Half,0,2700000,0,6500,Gwangju FC,18,Yool Heo,"[{'event_name': 'Passes Received', 'property':...",0.3413,0.7579,NaN,NaN,RIGHT
4,4,Half,1st Half,0,2700000,0,15021,Gwangju FC,18,Yool Heo,"[{'name': 'VHIR', 'property': {'duration': 500...",0.3500,0.5752,0.3501,0.5339,RIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433,1832,Half,2nd Half,1,2700000,2700000,5999333,Daegu FC,11,Cesinha,"[{'event_name': 'Passes Received', 'property':...",0.8861,0.1069,NaN,NaN,RIGHT
3434,1833,Half,2nd Half,1,2700000,2700000,6000600,Daegu FC,11,Cesinha,"[{'event_name': 'Passes', 'property': {'Outcom...",0.8691,0.1376,0.7890,0.2495,RIGHT
3435,1834,Half,2nd Half,1,2700000,2700000,6002367,Daegu FC,2,Jaewon Hwang,"[{'event_name': 'Crosses', 'property': {'Outco...",0.7890,0.2495,0.9670,0.6118,RIGHT
3436,1835,Half,2nd Half,1,2700000,2700000,6004200,Daegu FC,17,Italo Carvalho,"[{'event_name': 'Duels', 'property': {'Type': ...",0.9670,0.6118,NaN,NaN,RIGHT
